# Protein Mutation Tutorial for $\lambda$ Dynamics Workshop

## I. Introduction

### I.A. Free Energy Cycle

### I.B. Approximations

### I.C. Patches

#### I.C.1. Side Chain Patches

#### I.C.2. Whole Residue Patches for Proline and Glycine

## II. CHARMM-GUI Setup

### II.A. Patches

## III. Setting Up prep Directories

## IV. Running ALF

## V. Analyzing Results